In [1]:
import httpx 
import requests 
import pandas as pd 
from datetime import datetime, timedelta

In [102]:
expirations_url = "http://127.0.0.1:25510/v2/list/expirations?root=SPXW"
response = requests.get(expirations_url).json()
all_exps = response["response"]

today = datetime.today()
one_week_ago = today - timedelta(days=63)
recent_exps = [str(exp) for exp in all_exps if one_week_ago <= datetime.strptime(str(exp), "%Y%m%d") <= today]

In [103]:
base_url = "http://127.0.0.1:25510/v2/bulk_hist/option/quote"
df_list = []

for exp in recent_exps:
    params = {
        "root": "SPXW",
        "exp": exp,
        "start_date": exp,
        "end_date": exp,
        "ivl": 12000
    }
    r = requests.get(base_url, params=params)
    data = r.json()["response"]
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(exp)
    df_list.append(df)

In [104]:
df_all = pd.concat(df_list, ignore_index=True)
df_all['date'] = pd.to_datetime(df_all['date'])
df_all.set_index('date', inplace=True)

In [105]:
import pandas as pd

flattened_rows = []

for idx, row in df_all.iterrows():
    contract = row['contract']
    ticks = row['ticks']
    for tick in ticks:
        flattened_rows.append({
            'timestamp': tick[0],
            'bid_sz': tick[1],
            'bid_px': tick[3],
            'ask_sz': tick[5],
            'ask_px': tick[7],
            'strike': contract['strike'],
            'right': contract['right'],
            'expiration': contract['expiration'],
            'date': idx
        })

df_flat = pd.DataFrame(flattened_rows)

In [106]:
df_flat["date"] = pd.to_datetime(df_flat["date"], format="%Y%m%d")
df_flat["expiration"] = pd.to_datetime(df_flat["expiration"], format="%Y%m%d")

In [107]:
df_0dte = df_flat[df_flat['date'] == df_flat['expiration']].copy()

In [108]:
df_0dte["timestamp"] = df_0dte["timestamp"].apply(
    lambda x: (datetime.min + timedelta(milliseconds = x)).time()
)

In [109]:
df_auction = df_0dte[df_0dte["timestamp"] == "09:30:00"].copy()

In [110]:
from datetime import time 
target_time = time(9, 30)
time_lower = (datetime.combine(datetime.today(), target_time) - timedelta(minutes=5)).time()
time_upper = (datetime.combine(datetime.today(), target_time) + timedelta(minutes=5)).time()

df_auction = df_0dte[(df_0dte['timestamp'] >= time_lower) & (df_0dte['timestamp'] <= time_upper)].copy()

In [111]:
df_auction['mid_px'] = (df_auction['bid_px'] + df_auction['ask_px']) / 2
df_auction = df_auction[df_auction['mid_px'] > 0]

daily_atm_quotes = []

for date, group in df_auction.groupby('date'):
    group['spread'] = group['ask_px'] - group['bid_px']
    best_strike_row = group.loc[group['spread'].idxmin()]
    atm_strike = best_strike_row['strike']

    atm_call = group[(group['strike'] == atm_strike) & (group['right'] == 'C')]
    atm_put = group[(group['strike'] == atm_strike) & (group['right'] == 'P')]

    if not atm_call.empty and not atm_put.empty:
        call_mid = atm_call['mid_px'].values[0]
        put_mid = atm_call['mid_px'].values[0]

        daily_atm_quotes.append({
            'date': date,
            'strike': atm_strike,
            'call_mid': call_mid,
            'put_mid': put_mid,
            'call_bid': atm_call['bid_px'].values[0],
            'call_ask': atm_call['ask_px'].values[0],
            'put_bid': atm_put['bid_px'].values[0],
            'put_ask': atm_put['ask_px'].values[0],
        })

In [112]:
df_daily_quotes = pd.DataFrame(daily_atm_quotes).sort_values('date')

In [126]:
df_daily_quotes.to_parquet("/Users/kennethzhang/Desktop/VolAuction/data/auction_daily_quotes.parquet", index=False)
df_daily_quotes

,date,strike,call_mid,put_mid,call_bid,call_ask,put_bid,put_ask
0,2025-01-28,5930000,97.15,97.15,96.8,97.5,0.75,0.85
1,2025-01-29,5970000,90.40,90.40,90.0,90.8,0.95,1.00
2,2025-01-30,5990000,72.80,72.80,71.4,74.2,1.05,1.15
3,2025-01-31,1200000,4894.80,4894.80,4887.7,4901.9,0.00,0.05
4,2025-02-03,5835000,108.90,108.90,106.3,111.5,1.20,1.25
5,2025-02-04,5905000,92.95,92.95,90.8,95.1,1.25,1.30
6,2025-02-05,5945000,81.10,81.10,79.5,82.7,1.00,1.05
7,2025-02-06,6015000,64.10,64.10,61.1,67.1,0.85,0.95
8,2025-02-07,1400000,4686.05,4686.05,4678.1,4694.0,0.00,0.05
9,2025-02-10,5990000,70.45,70.45,70.2,70.7,1.05,1.10


In [120]:
df_auction

,timestamp,bid_sz,bid_px,ask_sz,ask_px,strike,right,expiration,date,mid_px
1,09:30:12,1,3618.8,1,3630.4,2400000,C,2025-01-28,2025-01-28,3624.60
2,09:30:24,1,3615.6,1,3627.1,2400000,C,2025-01-28,2025-01-28,3621.35
3,09:30:36,1,3613.1,1,3625.6,2400000,C,2025-01-28,2025-01-28,3619.35
4,09:30:48,1,3613.1,1,3627.2,2400000,C,2025-01-28,2025-01-28,3620.15
5,09:31:00,1,3616.7,1,3632.8,2400000,C,2025-01-28,2025-01-28,3624.75
...,...,...,...,...,...,...,...,...,...,...
22059722,09:34:12,1,534.3,1,546.1,4975000,C,2025-03-31,2025-03-31,540.20
22059723,09:34:24,1,532.3,1,533.0,4975000,C,2025-03-31,2025-03-31,532.65
22059724,09:34:36,1,523.5,1,544.3,4975000,C,2025-03-31,2025-03-31,533.90
22059725,09:34:48,1,530.0,1,541.6,4975000,C,2025-03-31,2025-03-31,535.80


In [121]:
atm_strike_map = {}
df_auction['spread'] = df_auction['ask_px'] - df_auction['bid_px']
for date, group in df_auction.groupby('date'):
    group = group[group['spread'] > 0]
    if group.empty:
        continue
    best_row = group.loc[group['spread'].idxmin()]
    best_strike = best_row['strike']
    atm_strike_map[date] = best_strike

In [125]:
pd.DataFrame([
    {'date': k, 'strike': v} for k, v in atm_strike_map.items()
]).to_csv("/Users/kennethzhang/Desktop/VolAuction/data/atm_strike_map.csv", index=False)

In [127]:
df_all_intraday = []

for date in df_flat['date'].unique():
    df_day = df_flat[df_flat['date'] == date]
    strike = atm_strike_map.get(date)

    if not strike:
        continue 

    df_atm = df_day[(df_day['strike'] == strike) & (df_day['expiration'] == date)] 
    df_atm['mid_px'] = (df_atm['bid_px'] + df_atm['ask_px']) / 2
    df_atm['time'] = pd.to_timedelta(df_atm['timestamp'], unit='ms')

    df_all_intraday.append(df_atm)

df_combined = pd.concat(df_all_intraday)
df_combined.to_parquet("/Users/kennethzhang/Desktop/VolAuction/data/all_intraday_quotes.parquet", index=False)

/var/folders/bc/0sv4l9h92yb5pz7_7vm4gh900000gn/T/ipykernel_93137/451892958.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_atm['mid_px'] = (df_atm['bid_px'] + df_atm['ask_px']) / 2
/var/folders/bc/0sv4l9h92yb5pz7_7vm4gh900000gn/T/ipykernel_93137/451892958.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_atm['time'] = pd.to_timedelta(df_atm['timestamp'], unit='ms')
/var/folders/bc/0sv4l9h92yb5pz7_7vm4gh900000gn/T/ipykernel_93137/451892958.py:11: SettingWithCopyWarning: 
A value is trying to be

In [129]:
df_combined

,timestamp,bid_sz,bid_px,ask_sz,ask_px,strike,right,expiration,date,mid_px,time
440926,34200000,0,0.0,0,0.00,5930000,C,2025-01-28,2025-01-28,0.000,0 days 09:30:00
440927,34212000,1,96.8,1,97.50,5930000,C,2025-01-28,2025-01-28,97.150,0 days 09:30:12
440928,34224000,1,93.7,1,94.30,5930000,C,2025-01-28,2025-01-28,94.000,0 days 09:30:24
440929,34236000,1,93.3,1,93.90,5930000,C,2025-01-28,2025-01-28,93.600,0 days 09:30:36
440930,34248000,3,94.9,1,95.70,5930000,C,2025-01-28,2025-01-28,95.300,0 days 09:30:48
...,...,...,...,...,...,...,...,...,...,...,...
21564396,57528000,0,0.0,8985,0.05,1200000,P,2025-03-31,2025-03-31,0.025,0 days 15:58:48
21564397,57540000,0,0.0,8985,0.05,1200000,P,2025-03-31,2025-03-31,0.025,0 days 15:59:00
21564398,57552000,0,0.0,8985,0.05,1200000,P,2025-03-31,2025-03-31,0.025,0 days 15:59:12
21564399,57564000,0,0.0,9128,0.05,1200000,P,2025-03-31,2025-03-31,0.025,0 days 15:59:24


In [131]:
from pathlib import Path 

output_path = Path("/Users/kennethzhang/Desktop/VolAuction/data/intraday_by_day")
output_path.mkdir(parents=True, exist_ok=True)

for date in df_flat['date'].unique():
    df_day = df_flat[df_flat['date'] == date]
    
    strike = atm_strike_map.get(date)
    if not strike:
        continue

    df_atm = df_day[(df_day['strike'] == strike) & (df_day['expiration'] == date)]
    df_atm['mid_px'] = (df_atm['bid_px'] + df_atm['ask_px']) / 2
    df_atm['time'] = pd.to_timedelta(df_atm['timestamp'], unit='ms')
    
    fname = output_path / f"intraday_quotes_{date.strftime('%Y-%m-%d')}.parquet"
    df_atm.to_parquet(fname, index=False)

/var/folders/bc/0sv4l9h92yb5pz7_7vm4gh900000gn/T/ipykernel_93137/2333841893.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_atm['mid_px'] = (df_atm['bid_px'] + df_atm['ask_px']) / 2
/var/folders/bc/0sv4l9h92yb5pz7_7vm4gh900000gn/T/ipykernel_93137/2333841893.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_atm['time'] = pd.to_timedelta(df_atm['timestamp'], unit='ms')
/var/folders/bc/0sv4l9h92yb5pz7_7vm4gh900000gn/T/ipykernel_93137/2333841893.py:14: SettingWithCopyWarning: 
A value is trying to